In [1]:
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split as sklearn_train_test_split
from sklearn.preprocessing import LabelEncoder
from scipy.sparse import csr_matrix

import pandas as pd
import numpy as np
import zipfile
import warnings
warnings.filterwarnings('ignore')

In [2]:
ratings = pd.read_csv('ratings.csv').drop(['Unnamed: 0'], axis=1)
providers = pd.read_csv('providers.csv').drop(['Unnamed: 0'], axis=1)
ratings

,User_id,Registration number,rating
0,0,2001,0.5
1,0,6860,2.2
2,0,7986,3.1
3,0,8023,4.7
4,0,6842,4.4
...,...,...,...
55512,24,5480,2.0
55513,24,1923,0.3
55514,24,4521,3.4
55515,24,5479,2.5


In [37]:
providers.columns = providers.columns.str.replace('index', 'Registration number')
providers

,Registration number,Регистрационный номер,Наименование,Вид деятельности/отрасль,Телефон,Предмет поставки,Важная информация,Сводный индикатор,"Уставный капитал, RUB",Вид деятельности/отрасль.1,Руководитель - ФИО
0,0,1142651006398,"1-АЯ ВАТЕР КОМПАНИ, ООО","Торговля оптовая соками, минеральной водой и п...",+7 (879) 3259919,NaN,NaN,Низкий риск,100000.0,"Торговля оптовая соками, минеральной водой и п...",Коптиевский Вадим Валерьевич
1,1,1080276000298,"108 ЧК, ООО","Торговля оптовая кофе, чаем, какао и пряностями",+7 (347) 2374074,NaN,На 24.05.2023 22:06 имеются действующие решени...,Средний риск,10000.0,"Торговля оптовая кофе, чаем, какао и пряностями",Мухамадиева Динара Фандусовна
2,2,1197746593835,"13 УСТРИЦ, ООО","Торговля оптовая рыбой, ракообразными и моллюс...",NaN,NaN,NaN,Низкий риск,25000.0,"Торговля оптовая рыбой, ракообразными и моллюс...",Буторина Екатерина Владимировна
3,3,1167847127953,"16-Я РЕСПУБЛИКА, ООО",Торговля оптовая неспециализированная пищевыми...,+7 (495) 0035916\n+7 (800) 1001681,NaN,NaN,Высокий риск,10000.0,Торговля оптовая неспециализированная пищевыми...,Жилина Галина Ивановна
4,4,1121103001018,"21 ВЕК, ООО",Торговля оптовая пивом,+7 (82151) 55207,NaN,NaN,Низкий риск,10000.0,Торговля оптовая пивом,Закирова Наталья Николаевна
...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,1177746445250,"ЯГОДЫ СИБИРИ, ООО",Торговля оптовая фруктами и овощами,+7 (905) 7703286,NaN,NaN,Высокий риск,300000.0,Торговля оптовая фруктами и овощами,Лебедева Олеся Олеговна
9996,9996,1042127014236,"ЯГУАР, ООО",Торговля оптовая напитками,+7 (8352) 632320\n+7 (8352) 635657\n+7 (8352) ...,NaN,На 19.03.2023 22:05 имеются действующие решени...,Высокий риск,10090000.0,Торговля оптовая напитками,Мухин Дмитрий Алексеевич
9997,9997,1159102132640,"ЯГУАР, ООО",Торговля оптовая фруктами и овощами,+7 (978) 1273228\n+7 (978) 7407340,01.24.29.110 Черешня\n01.25.31.000 Миндаль\n01...,NaN,Низкий риск,10000.0,Торговля оптовая фруктами и овощами,Буздаков Антон Александрович
9998,9998,1192130009026,"ЯДРИНСКИЙ КООПТОРГ, ООО",Торговля оптовая неспециализированная пищевыми...,+7 (960) 3065036,NaN,NaN,Низкий риск,250000.0,Торговля оптовая неспециализированная пищевыми...,Казамбаева Людмила Виссарионовна


In [38]:
## Создаем матрицу
def ratings_matrix(ratings):    
    # создайте csr матрицу 
    return csr_matrix(pd.crosstab(ratings['User_id'], ratings['Registration number'], ratings.rating, aggfunc=sum).fillna(0).values)  

R = ratings_matrix(ratings)

In [39]:
R

<25x9891 sparse matrix of type '<class 'numpy.float64'>'
	with 110761 stored elements in Compressed Sparse Row format>

1. Определим из всей выборки пользователе, которых будем сравнивать с активным
To find the 
 most similar users to 
, we use the cosine similarity and compute 
 for all 
. Fortunately, libraries such as scikit-learn (sklearn) are very useful for such tasks :

Создадим модель ближайших соседей из sklearn в функции create_model(). С кол-вом соседеней = 21, методом подбора "brute2.

Функция nearest_neighbors() возвращает knn пользователей для каждого пользователя.

In [40]:
def create_model(rating_matrix, metric):
    """
    - создание модели с базовыми параметрами
    """
    model = NearestNeighbors(metric=metric, n_neighbors=10, algorithm='brute')
    
    model.fit(rating_matrix)    
    return model
def nearest_neighbors(rating_matrix, model):
    """    
    :param rating_matrix : матрица рейтингов (nb_users, nb_items)
    :param model : модель knn  
    """    
    similarities, neighbors = model.kneighbors(rating_matrix)        
    return similarities[:, 1:], neighbors[:, 1:]

# метрику схожести успользуем Косинусную
model = create_model(rating_matrix=R, metric='cosine')
similarities, neighbors = nearest_neighbors(R, model)

2. Поиск элементов пользователя

In [41]:
def find_candidate_items(userid):
    """
    Поиск элементов для переданного пользователя
    
    :param userid : пользователь id
    :param neighbors : схожесть между пользователями      
    :return candidates : топ 10 элементов для пользователя
    """
    user_neighbors = neighbors[userid]
    
    activities = ratings.loc[ratings['User_id'].isin(user_neighbors)]
    
    # сортируем элементы по частоте
    frequency = activities.groupby('Registration number')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency['Registration number']
    active_items = ratings.loc[ratings['User_id'] == userid]['Registration number'].to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:10]
        
    return candidates

3. Предикт рейтинга
Для предикта необходимо:

Сходство между пользвателями, которое получаем из функции nearest_neighbors()
Нормализация рейтинга на общее среднее значение по пользователю 
.

In [42]:
# средний рейтинг по всем
mean = ratings.groupby(by='User_id', as_index=False)['rating'].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=('','_mean'), on='User_id')

# нормализация рейтинга
mean_ratings['norm_rating'] = mean_ratings['rating'] - mean_ratings['rating_mean']

mean = mean.to_numpy()[:, 1]
np_ratings = mean_ratings.to_numpy()

In [43]:
def predict(userid, itemid):
    """
    предикт для пользователя userid рейтинга на элемент itemid.
    
    :param
        - userid : пользователь для предикта
        - itemid : элемент для предикта
        
    :return
        - r_hat : предикт 
    """
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]
    
    # средний рейтинг
    user_mean = mean[userid]
    
    # поиск пользователей, которые имеют рейтинг по элементу 'itemid'
    iratings = np_ratings[np_ratings[:, 1].astype('int') == itemid]
    
    # поиск похожих пользователей
    simus = iratings[np.isin(iratings[:, 0], user_neighbors)]
    
    # отбор похожих пользователей, которые имеют рейтинг по выбранному элементу
    normalized_ratings = simus[:,4]
    indexes = [np.where(user_neighbors == uid)[0][0] for uid in simus[:, 0].astype('int')]
    sims = user_similarities[indexes]
    
    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))
    
    if num == 0 or den == 0:
        return user_mean
    
    # реализуем формулу предикта
    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))
    
    return r_hat

In [44]:
def user2userPredictions(userid, pred_path):
    """
    Сделаем предикт для каждого пользователя и сохраним в файл prediction.csv
    
    :param
        - userid : пользователя id
        - pred_path : куда сохраняем
    """    
    
    try:
        # поиск пользователей
        candidates = find_candidate_items(userid)

        # цикл по всем выбраным пользователям для предикта
        for itemid in candidates:

            # предикт для пользователя, по элементам
            r_hat = predict(userid, itemid)

            # сохраним
            with open(pred_path, 'a+') as file:
                line = '{},{},{}\n'.format(userid, itemid, r_hat)
                file.write(line)
    except IndexError:
        pass

In [45]:
import sys
import os

def user2userCF():
    """
    Предикт для всех пользователей, даже с 1 рейтингом   
    """
    # список всех пользователей
    users = ratings['User_id'].unique()
    
    def _progress(count):
        sys.stdout.write('\rRating predictions. Progress status : %.1f%%' % (float(count/len(users))*100.0))
        sys.stdout.flush()
    
    saved_predictions = 'predictions.csv'    
    if os.path.exists(saved_predictions):
        os.remove(saved_predictions)
    
    for count, userid in enumerate(users):        
        # делаем предикт
        user2userPredictions(userid, saved_predictions)
        _progress(count)

In [46]:
user2userCF()

Rating predictions. Progress status : 96.0%

4. Top-N рекомендаций
Функция user2userRecommendation() делает отбор необходимых рекомендаций для пользователя

In [47]:
def user2userRecommendation(userid):
    """
    Делаем предикт для пользователя
    """
    
    saved_predictions = 'predictions.csv'
    
    predictions = pd.read_csv(saved_predictions, sep=',', names=['User_id', 'Registration number', 'predicted_rating'])
    predictions = predictions[predictions['User_id']==userid]
    List = predictions.sort_values(by=['predicted_rating'], ascending=False)
    
    List = pd.merge(List, providers, on='Registration number', how='inner')
    
    return List

In [48]:
user2userRecommendation(4)

,User_id,Registration number,predicted_rating,Регистрационный номер,Наименование,Вид деятельности/отрасль,Телефон,Предмет поставки,Важная информация,Сводный индикатор,"Уставный капитал, RUB",Вид деятельности/отрасль.1,Руководитель - ФИО
0,4,566,2.484412,1152468000057,"АЛКОТРЭЙД 54, ООО",Торговля оптовая пивом,+7 (964) 5206555\n+7 (967) 6008000,NaN,NaN,Средний риск,10010000.0,Торговля оптовая пивом,Туленцев Александр Викторович
1,4,9605,2.341068,1157746326902,"ШТОРМ, ООО",Торговля оптовая мясом и мясными продуктами,+7 (495) 2017923\n+7 (916) 2036869\n+7 (916) 6...,10.11.31.140 Субпродукты пищевые крупного рога...,NaN,Низкий риск,30000.0,Торговля оптовая мясом и мясными продуктами,Тюрин Артём Сергеевич
2,4,3238,2.253015,1193702014758,"КАМЕЛИЯ, ООО","Торговля оптовая кофе, чаем, какао и пряностями",NaN,NaN,NaN,Средний риск,50000.0,"Торговля оптовая кофе, чаем, какао и пряностями",Самойлова Лада Германовна
3,4,2459,2.250081,1090280025439,"ДЕКАБРЬ, ООО","Торговля оптовая сахаром, шоколадом и сахарист...",+7 (917) 4043045\n+7 (3472) 660188\n+7 (3472) ...,"10.82.23.149 Конфеты, глазированные помадой, с...",NaN,Низкий риск,10000.0,"Торговля оптовая сахаром, шоколадом и сахарист...",Егорова Алина Ринатовна
4,4,5633,2.247162,1166820063618,"ОСИРИС, ООО",Торговля оптовая пищевыми маслами и жирами,+7 (475) 2710595\n+7 (926) 8853797,NaN,NaN,Высокий риск,9000000.0,Торговля оптовая пищевыми маслами и жирами,Белков Владимир Владимирович
5,4,3838,2.153345,1022301196246,"КУБАНСКАЯ ГРУППА, ООО","Торговля оптовая рыбой, ракообразными и моллюс...",+7 (8617) 762529\n+7 (988) 7623444,"10.13 Продукция мясная пищевая, в том числе из...",NaN,Низкий риск,50000.0,"Торговля оптовая рыбой, ракообразными и моллюс...",Мацарета Иван Федорович
6,4,272,1.877915,1196820002059,"АГРОТРЕЙДЕР, ООО","Торговля оптовая мясом и мясом птицы, включая ...",+7 (962) 2363615,NaN,NaN,Средний риск,10000.0,"Торговля оптовая мясом и мясом птицы, включая ...",Копылов Александр Геннадьевич
7,4,7720,1.860946,1115190025686,"СТЕЛЛА, ООО","Торговля оптовая рыбой, ракообразными и моллюс...",+7 (815) 2472490\n+7 (815) 2473253\n+7 (921) 0...,NaN,На 21.02.2022 17:00 имеются действующие решени...,Высокий риск,20000.0,"Торговля оптовая рыбой, ракообразными и моллюс...",Джаран Игорь Викторович
8,4,4653,1.815863,1104703002084,"МИР МОЛОКА СПБ, ООО",Деятельность агентов по оптовой торговле пищев...,NaN,NaN,NaN,Низкий риск,10000.0,Деятельность агентов по оптовой торговле пищев...,Голосова Марина Сергеевна
9,4,3778,1.511136,5077746773000,"КРОХА, ООО",Торговля оптовая гомогенизированными пищевыми ...,+7 (495) 9357010\n+7 (916) 3100508,NaN,На 09.02.2022 22:23 имеются действующие решени...,Высокий риск,1000000.0,Торговля оптовая гомогенизированными пищевыми ...,Калинина Любовь Борисовна
